In [2]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import matplotlib.pylab as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from utilities import *

DATA = Path('.').resolve().parent / 'data'

# Question 1 (6 points) 
Run a discriminant analysis with both predictors using the entire dataset as training data. 
Among those who completed the tasks, what is the percentage of administrators who are classified incorrectly as failing to complete the tasks? 

In [3]:
sysadmin_df = pd.read_csv(DATA / 'SystemAdministrators.csv')
sysadmin_df.columns = [s.strip().replace(' ', '_') for s in sysadmin_df.columns]
sysadmin_df.head()

,Experience,Training,Completed_task
0,10.9,4,Yes
1,9.9,4,Yes
2,10.4,6,Yes
3,13.7,6,Yes
4,9.4,8,Yes


In [4]:
scaleInput = MinMaxScaler()
X = scaleInput.fit_transform(sysadmin_df.drop(columns=['Completed_task']))
labelOutput = LabelEncoder()
y = labelOutput.fit_transform(sysadmin_df['Completed_task'])

In [5]:
da_clf = LinearDiscriminantAnalysis()
da_clf.fit(X, y)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [6]:
train_predict = da_clf.predict(X)
y_predict = labelOutput.inverse_transform(train_predict)

C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
printConfusionMatrix(sysadmin_df['Completed_task'], y_predict)

Confusion Matrix (Accuracy 0.9067)

         Prediction
Reference  0  1
        0 58  2
        1  5 10


In [8]:
print('{:.1%}'.format(5/15), 'administrators who are classified incorrectly as failing to complete the tasks') 

33.3% administrators who are classified incorrectly as failing to complete the tasks


# Question 2 (4 points)
Compute the two classification scores (the "task completed" classification score and the "task not completed" classification score) for an administrator with four months of experience and six credits of training. is this administrator classified as "task not completed" or "task completed"? 

In [9]:
newAdmin = pd.DataFrame([{'Experience' : 4, 'Training' : 6}])
X_newAdmin = scaleInput.fit_transform(newAdmin)

In [10]:
print('the "task completed" classification score=',da_clf.predict_proba(X_newAdmin)[:, 0])
print('the "task not completed" classification score=',da_clf.predict_proba(X_newAdmin)[:, 1])

the "task completed" classification score= [0.99989136]
the "task not completed" classification score= [0.00010864]


this administrator is classified as "task not completed"

# Question 3 (10 points)
Now partition the original data into training and validation data (set the random seed to 12345), and run a discriminant analysis, AND a neural net. For each method, compare the training and validation results, and comment. 

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=12345)
da_clf = LinearDiscriminantAnalysis()
da_clf.fit(X_train, y_train)
train_predict = da_clf.predict(X_train)
valid_predict = da_clf.predict(X_valid)
printConfusionMatrix(labelOutput.inverse_transform(y_train), labelOutput.inverse_transform(train_predict))
printConfusionMatrix(labelOutput.inverse_transform(y_valid), labelOutput.inverse_transform(valid_predict))

Confusion Matrix (Accuracy 0.9111)

         Prediction
Reference  0  1
        0 33  1
        1  3  8
Confusion Matrix (Accuracy 0.8667)

         Prediction
Reference  0  1
        0 24  2
        1  2  2


C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

In [12]:
car_nnet = MLPClassifier(hidden_layer_sizes=(4,), activation='logistic', solver='lbfgs', random_state=1)
car_nnet.fit(X_train, y_train)
validPrediction = car_nnet.predict(X_valid)
trainPrediction = car_nnet.predict(X_train)
printConfusionMatrix(labelOutput.inverse_transform(y_train), labelOutput.inverse_transform(trainPrediction))
printConfusionMatrix(labelOutput.inverse_transform(y_valid), labelOutput.inverse_transform(validPrediction))

Confusion Matrix (Accuracy 0.9556)

         Prediction
Reference  0  1
        0 33  1
        1  1 10
Confusion Matrix (Accuracy 0.8667)

         Prediction
Reference  0  1
        0 23  3
        1  1  3


C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

We can see use neural nets, the training accuracy increases a lot comparing to discriminant analysis, but the validation set accuracy doesn't change much.